In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Load SST-2 dataset
dataset = load_dataset("aayya/sst2-augmented")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

# View a sample
for _, example in zip(range(5), dataset["train_orig"]):
  print(example["sentence"], example["label"])

hide new secretions from the parental units  0
contains no wit , only labored gags  0
that loves its characters and communicates something rather beautiful about human nature  1
remains utterly satisfied to remain the same throughout  0
on the worst revenge-of-the-nerds clichés the filmmakers could dredge up  0


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string

stop_words = set(stopwords.words('english'))
def preproc(text: str) -> str:
  text = str(text).lower()                            # Lowercase words
  text = re.sub(r"\[(.*?)\]", "", text)               # Remove [+XYZ chars] in content
  text = re.sub(r"\s+", " ", text)                    # Remove multiple spaces in content
  text = re.sub(r"\w+…|…", "", text)                  # Remove ellipsis (and last word)
  text = re.sub(r"(?<=\w)-(?=\w)", " ", text)         # Replace dash between words
  text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Remove punctuation

  tokens = word_tokenize(text)                                            # Get tokens from text
  tokens = [t for t in tokens if not t in stop_words]                  # Remove stopwords
  tokens = ["" if t.isdigit() else t for t in tokens]                 # Remove digits
  tokens = [t for t in tokens if len(t) > 1]                          # Remove short tokens
  return " ".join(tokens)

In [ ]:
for _, example in zip(range(5), dataset["train_orig"]):
  print(example["sentence"], example["label"])
  print(preproc(example["sentence"]))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import numpy as np

In [ ]:
train_sentences, train_labels = [], []
for example in dataset["train_orig_eda_embedding_wordnet"]:
  train_sentences.append(preproc(example["sentence"]))
  train_labels.append(int(example["label"]))

In [ ]:
test_sentences, test_labels = [], []
for example in dataset["val"]:
  test_sentences.append(preproc(example["sentence"]))
  test_labels.append(int(example["label"]))

In [ ]:
train_labels[:5]

In [ ]:
train_sentences[:5]

In [ ]:
nb_pip = Pipeline([
    # ("vect", CountVectorizer()),
    ("tf-idf", TfidfVectorizer()),
    ("nb", MultinomialNB()),
])

nb_pip.fit(train_sentences, train_labels)
acc = np.mean(nb_pip.predict(test_sentences) == test_labels)


In [ ]:
nb_pip = Pipeline([
    # ("vect", CountVectorizer()),
    ("tf-idf", TfidfVectorizer()),
    ("nb", BernoulliNB()),
])

nb_pip.fit(train_sentences, train_labels)
acc = np.mean(nb_pip.predict(test_sentences) == test_labels)


In [ ]:
test_results = nb_pip.predict(test_sentences)
tp = np.sum((test_results == 1) & (np.array(test_labels) == 1))
fp = np.sum((test_results == 1) & (np.array(test_labels) == 0))
tn = np.sum((test_results == 0) & (np.array(test_labels) == 0))
fn = np.sum((test_results == 0) & (np.array(test_labels) == 1))

acc = (tp + tn) / (tp + fp + tn + fn)
prec = tp / (tp + fp)
rec = tp / (tp + fn)
f1 = 2 * prec * rec / (prec + rec)

In [ ]:
acc, prec, rec, f1

In [ ]:
corpus = [preproc(example["sentence"]) for example in dataset["train_orig"]]

In [ ]:
corpus_ = [" ".join(example) for example in corpus]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(train_sentences)
vectorizer.get_feature_names_out()

In [ ]:
len(dataset["train_orig"])

In [ ]:
X.shape

In [ ]:
from sklearn.cluster import KMeans

tp = fp = tn = fn = 0

for _ in range(10):
  km = Pipeline([
      ("tf-idf", TfidfVectorizer()),
      ("kmeans", KMeans(n_clusters=2, n_init="auto")),
  ])
  km.fit(train_sentences, train_labels)
  test_results = km.predict(test_sentences)

  tp += np.sum((test_results == 1) & (np.array(test_labels) == 1))
  fp += np.sum((test_results == 1) & (np.array(test_labels) == 0))
  tn += np.sum((test_results == 0) & (np.array(test_labels) == 0))
  fn += np.sum((test_results == 0) & (np.array(test_labels) == 1))


acc = (tp + tn) / (tp + fp + tn + fn)
prec = tp / (tp + fp)
rec = tp / (tp + fn)
f1 = 2 * prec * rec / (prec + rec)

acc, prec, rec, f1

In [ ]:
from sklearn.cluster import MiniBatchKMeans

km = MiniBatchKMeans(n_clusters=2).fit(X)
km.labels_
acc = (km.labels_ == [example["label"] for example in dataset["train_orig"]]).mean()
print(f"Accuracy: {acc}")

In [ ]:
from sklearn.cluster import MeanShift

km = MeanShift().fit(X)
km.labels_

In [ ]:
from sklearn.cluster import MiniBatchKMeans

mkm = MiniBatchKMeans(n_clusters=2).fit(X)
mkm.labels_
acc = (mkm.labels_ == [example["label"] for example in dataset["train"]]).mean()
print(f"Accuracy: {acc}")

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

db = DBSCAN().fit(X)
np.max(db.labels_)
acc = (db.labels_ == [example["label"] for example in dataset["train"]]).mean()
print(f"Accuracy: {acc}")

In [ ]:
!pip install gensim

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [ ]:
model = api.load("fasttext-wiki-news-subwords-300")

In [ ]:
w2v = model

In [ ]:
w2v.vector_size

In [ ]:
model.most_similar("movie")

In [ ]:
trains = []
for sent in train_sentences:
  vecs = []
  for word in sent.split():
    if word in model:
      vecs.append(model[word])

  avg = sum(vecs)/len(vecs) if len(vecs) > 0 else np.zeros(model.vector_size)
  trains.append(avg)
X = np.array(trains)

In [ ]:
X.shape

In [ ]:
X[0]

In [ ]:
from sklearn.neighbors import NearestCentroid
nc_pip = Pipeline([
    # ("vect", CountVectorizer()),
    ("tf-idf", TfidfVectorizer()),
    ("nc", NearestCentroid(n_jobs=1)),
])

nc_pip.fit(train_sentences, train_labels)

In [ ]:
np.mean(nc_pip.predict(test_sentences) == test_labels)